# Download YouTube Videos
**Author:** Scott Campit

This notebook adpted code from Yan Gobeil's [Medium article](https://towardsdatascience.com/making-an-image-dataset-from-youtube-videos-5116252d20a3) convert YouTube video clips into images.

The following modifications were made to ensure the code ran in Python 3.8+ with the latest dependency versions (which can be found in the `requirements.txt` file):

  1. Replaced Beautiful Soup with Selenium to get urls due to JavaScript file being pulled.
  2. Resolved several errors that popped up during the run. 

## Import Libraries
First, we need to install some important libraries.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from pytube import YouTube
import cv2
import os
import glob

## Setting up functions for downloading YouTube videos as images

### Get video URLs
Next, we'll define the first function, which searches a word or phrase on Youtube and returns urls for the videos.

In [3]:
def get_urls(query, site_to_query="https://www.youtube.com/", wait_duration=5, path_to_chrome_driver=r'C:/Users/Scott/Software/Browser_Utils/chromedriver.exe'):
    '''
    Search Youtube based on query and return a list of links to the videos resulting from the search. A maximum of results can be set (default 10).

    INPUTS
        * query: a string denoting the keyword or phrase to search on YouTube.
        * wait_duration: an integer denoting the amount of time to wait per click. This prevents us from getting into trouble by Google. The default value is 5s.
        * path_to_chrome_driver: a string denoting the path to your Chrome driver (needed to query on the Chrome browser using Selenium).

    OUTPUT
        * urls: a list of urls corresponding to the query.
    '''
    
    # Initialize on Chrome 
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(executable_path=path_to_chrome_driver, chrome_options=options)
    
    # Search on YouTube
    driver.get(site_to_query)
    WebDriverWait(driver, wait_duration).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input#search"))).send_keys(query)
    driver.find_element_by_css_selector("button.style-scope.ytd-searchbox#search-icon-legacy").click()
    
    # Get URLs/
    urls = [my_href.get_attribute("href") for my_href in WebDriverWait(driver, wait_duration).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.ytd-video-renderer#video-title")))]
    return urls

### Download images to local
Next, we'll download the videos to your local system using [pytube](https://python-pytube.readthedocs.io/en/latest/). 

In [6]:
def download_video(url, path=None):
    """
    download_video downloads all YouTube videos from the list of urls using pytube.
    
    INPUTS:
        * url: a list of urls correspond to the query.
    """
    try:
        yt = YouTube(url)
        
        yt = yt.streams.filter(file_extension='mp4').first()
        out_file = yt.download(path)
        file_name = out_file.split("\\")[-1]
        print(f"Downloaded {file_name} correctly!")
    except Exception as exc:
        print(f"Download of {url} did not work because of {exc}...")

The next bit of code will test the `download_video` function.

In [7]:
tmp = download_video(url=url)

Downloaded Total Thyroidectomy and Central Neck Dissection.mp4 correctly!


### Extract images from video
Finally, this function will extract images using [OpenCV](https://opencv.org/).

In [12]:
def extract_images_from_video(video, folder="D:/Data/InterOp/", delay=30, name="file", max_images=20, is_silent=True):
    """
    extract_images_from_video turns a YouTube video (.mp4) into a series of images using the opencv project. 

    INPUTS:
        * video
        * folder
        * delay
        * name
        * max_images
        * is_silent
    """    
    vidcap = cv2.VideoCapture(video)
    count = 0
    num_images = 0
    if not folder:
        folder = os.getcwd()
    label = 0
    success = True
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    
    try:
        while success and num_images < max_images:
            success, image = vidcap.read()
            num_images += 1
            label += 1
            file_name = name + "_" + str(label) + ".jpg"
            path = os.path.join(folder, file_name)
            cv2.imwrite(path, image)
            if cv2.imread(path) is None:
                os.remove(path)
            else:
                if not is_silent:
                    print(f'Image successfully written at {path}')
            count += delay*fps
            vidcap.set(1, count)
    except Exception as exc:
        print(f"Image separation stopped because of {exc}...")
        pass

# Putting it all together
Now let's aggregate all the functions we just developed into a single function to call for downloading images based on YouTube queries.


In [20]:
def extract_images_from_word(query, delete_video=True, image_delay=5, 
                             max_images=30, savepath="D:/Data/InterOp/Thyroidectomy/", silent=True):
    """
    """
    if not os.path.exists(savepath):
        os.mkdir(savepath)
    
    # Grab some URLs based on a query term
    urls = get_urls(query)
    
    # Download videos using the URLs we just extracted
    for url in urls:
        download_video(url)
    
    # Find all .mp4s and create images from them.
    for i, video in enumerate(glob.glob("*.mp4")):
        extract_images_from_video(video, 
                                  folder=savepath, 
                                  delay=image_delay, 
                                  name=query+'_'+str(i), 
                                  max_images=max_images, 
                                  silent=silent)
        
        # Save on memory when possible.
        if delete_video:
            os.remove(video)
    
    print("Finished extracting images from videos!")

## Extracting Thyroidectomy videos
Now, let's download some Thyroidectomy videos from YouTube. Gross.

In [17]:
extract_images_from_word(query="thyroidectomy", delete_video=True, image_delay=5, 
                            max_images=50, savepath="D:/Data/InterOp/Thyroidectomy/", silent=False)

.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_14.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_15.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_16.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_17.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_18.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_19.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_20.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_21.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_22.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_23.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidectomy_14_24.jpg
Image successfully written at D:/Data/InterOp/Thyroidectomy/thyroidecto